# PSG Complete Data Analysis - FootballDecoded

### Objetivo del Análisis

Este notebook realiza una **extracción exhaustiva y organizada** de todos los datos disponibles del **Paris Saint-Germain** para las temporadas **2023/24 y 2024/25**, utilizando el conjunto completo de wrappers desarrollados en FootballDecoded.

**Datos a extraer:**
- Estadísticas de equipo (temporada completa por competición)
- Estadísticas individuales (todos los jugadores de la plantilla)
- Eventos de partidos (goles, pases, disparos, etc.)
- Datos espaciales (coordenadas, mapas de calor, redes de pase)
- Métricas avanzadas (xG, xA, PPDA, build-up chains)
- Análisis específico del partido PSG vs Auxerre (01/09/2024)

**Fuentes de datos:**
- **FBref**: Estadísticas completas y eventos
- **Understat**: Métricas avanzadas (xGChain, PPDA, etc.)
- **WhoScored**: Datos espaciales y coordenadas

In [57]:
# Dependencias base
import pandas as pd
import numpy as np
from typing import Dict, List, Optional, Union
from datetime import datetime
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

import sys
import os
    
# Ir al directorio correcto donde está instalado el paquete
data_dir = "/home/oriol/FD/Data"
sys.path.insert(0, data_dir)
    
from scrappers import FBref, Understat, WhoScored
from wrappers import *

In [ ]:
# PSG 2023-24
psg_23_24 = fbref_extract_league_players(
    league="FRA-Ligue 1",
    season="2023-24", 
    team_filter="Paris S-G",
    verbose=True
)

# PSG 2024-25  
psg_24_25 = fbref_extract_league_players(
    league="FRA-Ligue 1",
    season="2024-25",
    team_filter="Paris S-G", 
    verbose=True
)

In [ ]:
# Extract data for 2023-24 players
psg_data_23_24 = fbref_extract_multiple_players(
    psg_23_24['player'].tolist(),
    "FRA-Ligue 1", 
    "2023-24",
    verbose=True
)

print("\n" + "="*50 + "\n")

# Extract data for 2024-25 players  
psg_data_24_25 = fbref_extract_multiple_players(
    psg_24_25['player'].tolist(),
    "FRA-Ligue 1", 
    "2024-25",
    verbose=True
)

In [ ]:
# Use existing player lists from FBref
players_23_24 = psg_23_24['player'].tolist()
players_24_25 = psg_24_25['player'].tolist()

# Extract Understat data for 2023-24 players
psg_understat_23_24 = understat_extract_multiple_players(
    players_23_24,
    "FRA-Ligue 1", 
    "2023-24",
    verbose=True
)

print("\n" + "="*50 + "\n")

# Extract Understat data for 2024-25 players  
psg_understat_24_25 = understat_extract_multiple_players(
    players_24_25,
    "FRA-Ligue 1", 
    "2024-25",
    verbose=True
)

In [71]:
# Merge 2023-24 data
psg_complete_23_24 = pd.merge(
    psg_data_23_24, 
    psg_understat_23_24,
    on=['player_name', 'league'],
    how='left',
    suffixes=('', '_dup')
)

# Remove duplicate columns
dup_cols = [col for col in psg_complete_23_24.columns if col.endswith('_dup')]
psg_complete_23_24 = psg_complete_23_24.drop(columns=dup_cols)

# Merge 2024-25 data
psg_complete_24_25 = pd.merge(
    psg_data_24_25, 
    psg_understat_24_25,
    on=['player_name', 'league'],
    how='left',
    suffixes=('', '_dup')
)

# Remove duplicate columns
dup_cols = [col for col in psg_complete_24_25.columns if col.endswith('_dup')]
psg_complete_24_25 = psg_complete_24_25.drop(columns=dup_cols)